#Blending
* voting ensemble
* voting ensemble with weights
* CVも計算

[2-methods-how-to-ensemble-predictions.ipynb](https://www.kaggle.com/code/radek1/2-methods-how-to-ensemble-predictions)


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
class CFG:

  ensemble_exp = '1'
  
  #ensemble csv
  path_1 = 'Candidate ReRank Model - [LB 0.575]_submission.csv' #クリスbaseline : 0.575
  path_2 = 'otto-pipeline2 [LB 0.576]_submission.csv' #best score 公開コード : 0.576
  path_3 = 'otto_pipeline2_LB0576_and_w2vec_ver4_submission.csv' #best score 公開コード + word2vec ver4  : 0.576
  
  #csv weight
  weight_1 = 1
  weight_2 = 1
  weight_3 = 1

In [3]:
import numpy as np
import pandas as pd
import os

In [4]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/output/chris_baseline/'
OUTPUT_DIR = f'/content/drive/MyDrive/kaggle/2022/OTTO/output/exp/ensemble/{CFG.ensemble_exp}/'

if not os.path.exists(OUTPUT_DIR):
  os.makedirs(OUTPUT_DIR)

#アンサンブル対象

In [5]:
path_1 = INPUT_DIR + CFG.path_1
path_2 = INPUT_DIR + CFG.path_2
path_3 = INPUT_DIR + CFG.path_3

In [6]:
paths = [path_1, path_2, path_3]

# Loading the data

In [7]:
!pip install polars # why are we using polars? it has much smaller memory footprint than pandas!

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import polars as pl

In [9]:
def read_sub(path, weight=1): # by default let us assing the weight of 1 to predictions from each submission, this will be akin to a standard vote ensemble
    '''a helper function for loading and preprocessing submissions'''
    return (
        pl.read_csv(path)
            .with_column(pl.col('labels').str.split(by=' '))
            .with_column(pl.lit(weight).alias('vote'))
            .explode('labels')
            .rename({'labels': 'aid'})
            .with_column(pl.col('aid').cast(pl.UInt32)) # we are casting the `aids` to `Int32`! memory management is super important to ensure we don't run out of resources
            .with_column(pl.col('vote').cast(pl.UInt8))
    )

In [10]:
%%time
#実験のaid投票

#重みなし
#subs = [read_sub(path) for path in paths]

#重みあり
subs = [read_sub(path, weight) for path, weight in zip(paths, [CFG.weight_1, CFG.weight_2, CFG.weight_3])]
subs[0].head()

CPU times: user 37.4 s, sys: 49.5 s, total: 1min 26s
Wall time: 1min


session_type,aid,vote
str,u32,u8
"""12899779_click...",59625,1
"""12899779_click...",1253524,1
"""12899779_click...",737445,1
"""12899779_click...",438191,1
"""12899779_click...",731692,1


In [11]:
%%time
#実験ごとのaid投票。joinじゃないとメモる落ちる？
subs = subs[0].join(subs[1], how='outer', on=['session_type', 'aid']).join(subs[2], how='outer', on=['session_type', 'aid'], suffix='_right2')
subs.head()

CPU times: user 4min 46s, sys: 1min 25s, total: 6min 12s
Wall time: 3min 16s


session_type,aid,vote,vote_right,vote_right2
str,u32,u8,u8,u8
"""12899779_click...",59625,1,1,1
"""12899779_click...",1253524,1,1,1
"""12899779_click...",737445,1,1,1
"""12899779_click...",438191,1,1,1
"""12899779_click...",731692,1,1,1


In [12]:
%%time
#merge

subs = (subs
    .fill_null(0)
    .with_column((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2')).alias('vote_sum'))
    .drop(['vote', 'vote_right', 'vote_right2'])
    .sort(by='vote_sum')
    .reverse()
)

subs.head()

CPU times: user 21.2 s, sys: 11.9 s, total: 33.1 s
Wall time: 14.5 s


session_type,aid,vote_sum
str,u32,u8
"""14571581_carts...",1497245,3
"""14571581_carts...",984794,3
"""14571581_carts...",1072049,3
"""14571581_carts...",1392029,3
"""14571581_carts...",1124107,3


In [13]:
%%time
#最初の 20 個の予測を取得session_typeして、スペースで区切られた文字列に変換

preds = subs.groupby('session_type').agg([
    pl.col('aid').head(20).alias('labels')
])

preds = preds.with_column(pl.col('labels').apply(lambda lst: ' '.join([str(aid) for aid in lst])))
preds

CPU times: user 6min 28s, sys: 11.1 s, total: 6min 39s
Wall time: 6min 9s


session_type,labels
str,str
"""13547481_order...","""1357380 712483..."
"""13675132_carts...","""1421005 163722..."
"""13139260_carts...","""799923 1005645..."
"""13867980_carts...","""571215 409717 ..."
"""13678366_carts...","""982423 1263710..."
"""13283461_carts...","""133322 963290 ..."
"""14475308_order...","""1426451 409620..."
"""14417300_order...","""95003 738832 8..."
"""13972932_order...","""1022572 137514..."


#CV recall@20

In [14]:
#pandas変更
#valid label読み込み
#CV計算のrecall

#sub.csv save 

In [17]:
%%time

preds.write_csv(OUTPUT_DIR + f'ensemble_{CFG.ensemble_exp}_submission.csv')

CPU times: user 1.81 s, sys: 1.1 s, total: 2.91 s
Wall time: 3.67 s
